In [1]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import exc, create_engine
from bs4 import BeautifulSoup
from urllib.request import urlopen
import logging

In [2]:
def sql_connection():
    try:
        connection = create_engine('postgresql+psycopg2://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + '5432' + '/' + os.environ.get('RDS_DB'),
                                    connect_args = {'options': '-csearch_path=nba_source'},
                     echo = False)
        logging.info('SQL Connection Successful')
        print('SQL Connection Successful')
        return(connection)
    except exc.SQLAlchemyError as e:
        logging.info('SQL Connection Failed, Error:', e)
        print('SQL Connection Failed, Error:', e)
        return(e)

In [3]:
conn = sql_connection()

SQL Connection Successful


In [12]:
tables = pd.read_sql_query('SELECT table_name FROM information_schema.tables', conn)


In [2]:
conn = psycopg2.connect(host=os.environ.get('IP'), 
                 port="5432", 
                 user=os.environ.get('RDS_USER'), 
                 password=os.environ.get('RDS_PW'), 
                 database=os.environ.get('RDS_DB'), 
                 options="-c search_path=nba_prod") # setting schema

In [6]:
df = pd.read_sql_query('SELECT table_name FROM information_schema.tables', conn)

In [8]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df2 = pd.DataFrame(data=d)
df2

,col1,col2
0,1,3
1,2,4


In [6]:
def write_to_sql(data, table_type):
    data_name = [ k for k,v in globals().items() if v is data][0]
    if len(data) == 0:
        print(data_name + " Failed, not writing to SQL")
    else:
        # ^ this disgusting monstrosity is to get the name of the -fucking- dataframe lmfao
        data.to_sql(con = conn, name = ("aws_" + data_name + "_table"), index = False, if_exists = table_type)
        print("Writing aws_" + data_name + "_table to SQL")

In [13]:
df.to_sql(con = conn, name = 'aws_stats_table_prac', index = False, if_exists = 'replace')